# Stock Price Prediction Models

This notebook features example code for every model used in the stock price prediction project, including statistical and AI-based approaches.

**Important Notes for Running this Notebook:**

*   **Environment:** This notebook is designed to run in Google Colab, which is a free cloud-based environment. Running locally may not work without significant setup.
*   **Hardware Acceleration:** I strongly recommend connecting to the **T4 GPU runtime** immediately (Runtime -> Change runtime type -> T4 GPU). The AI models later in the notebook are computationally intensive and will not run in a reasonable time on a CPU. Changing the runtime after loading data will require you to re-run the data acquisition section.
*   **Execution Time:** Running all cells in this notebook takes approximately 90 minutes on the T4 GPU.
*   **Data Dependency:** Please ensure you run the first section ("Acquiring data") completely, as it provides the necessary data for all subsequent sections and models.

# 1. Acquiring data

Define the list of tickers for which OHLCV data will be downloaded.

In [ ]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
# This website contains information about every security listed on the NASDAQ.
data_clean = data[data['Test Issue'] == 'N']

# Load 1 of the 3 options by uncommenting a line. Third option provides sufficient data for all of this notebook.

# symbols = data_clean['NASDAQ Symbol'].tolist() # Complete NASDAQ set. Not needed for this notebook.
# symbols = ['AAPL', 'JPM', 'NVDA', 'SPY'] # The 4 securities sufficient for predictions apart from GNNs and TFTGNN.
symbols = ['AAPL', 'JPM', 'NVDA', 'GOOGL', 'MSFT', 'AMZN', 'META',
           'TSLA', 'SONY', 'JBL', 'NFLX',  'AMD', 'TSM', 'ORCL', 'AVGO',
           'V', 'MS', 'COF', 'MET', 'AMAT', 'MCHP', 'ADI',
           'BAC', 'BLK', "ASML", 'QCOM', 'CRM', 'ADBE', 'GS', 'WFC',
           'SCHW', 'BK', 'AXP', 'TXN', 'MU', 'NXPI', 'KLAC', 'LRCX',
           'UNH', 'JNJ', 'PFE', 'MRK', 'LLY', 'TMO', 'BMY', 'LMT', 'F',
           'GM', 'HMC', 'TM', 'WMT', 'HD', 'COST', 'PG', 'KO', 'MCD',
           'TGT', 'PEP', 'JNJ', 'NVO', 'SPY', 'QQQ', 'DIA', 'IWM', 'XLK', 'XLF',
           'XLE', 'XLI', 'XLV', 'XLY', 'XLP', 'VNQ', 'IYR', 'VGT', 'VTI', 'VUG',
           'VTV', 'IWF', 'IWD', 'ITOT'] # These 80 securities are necessary to run the GNN and TFT-GNN models.

print('total number of symbols traded = {}'.format(len(symbols)))

Install yfinance:

In [ ]:
! pip install yfinance > /dev/null 2>&1

Use yfinance to acquire the data for each stock in the symbols dataframe. Downloading the recommended 80 tickers should work in one go. To download data for the full set of NASDAQ securities, you will hit Yahoo Finance API limits and have to do it in chunks.

In [ ]:
import yfinance as yf
import os, contextlib

offset = 0
limit = 1000
period = "max" # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

limit = limit if limit else len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = symbols[i]
            data = yf.download(s, period=period)
            if len(data.index) == 0:
                continue

            is_valid[i] = True
            data.to_csv('{}.csv'.format(s))

print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))

# 2. Statistical Models

## 2.1 SARIMA

Full year blind prediction, currently configured for AAPL 2018.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from itertools import product
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')
import gc

# Read CSV skipping the first two metadata rows
df = pd.read_csv('AAPL.csv', skiprows=3, header=None) # You can change AAPL to any of the loaded tickers.

# Set proper column names
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.set_index('Date', inplace=True)

train = df.loc['2015-01-01':'2017-12-31']['Close']
test = df.loc['2018-01-01':'2018-12-31']['Close']

# Model and Hyperparameters
model = SARIMAX(train, order=(2,1,3), seasonal_order=(1,1,1,22), enforce_stationarity=False, enforce_invertibility=False)
results = model.fit(disp=False)
forecast_result = results.get_forecast(steps=len(test))
forecast = forecast_result.predicted_mean
forecast_ci = forecast_result.conf_int() # confidence interval

# Set the index of forecast_ci to match the index of the test data
forecast_ci.index = test.index

plt.figure(figsize=(10,4))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, forecast, label='Forecast')
plt.fill_between(forecast_ci.index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='pink')
plt.legend()
plt.title('SARIMA Forecast')
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.show()

# Metrics
rmse = np.sqrt(mean_squared_error(test, forecast))
mae = mean_absolute_error(test, forecast)
mape = mean_absolute_percentage_error(test, forecast)
r2 = r2_score(test, forecast)
print(f'RMSE: {rmse:.2f}, MAE: {mae:.2f}', f'MAPE: {mape:.2f}', f'R2: {r2:.2f}')

Weekly sliding-window predictions. Note that this snippet takes a 5-15 minutes to run. Currently configured for AAPL 2021.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from joblib import Parallel, delayed
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# -------------------------------
# Load and preprocess data
# -------------------------------
df = pd.read_csv('AAPL.csv', skiprows=3, header=None)
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

series = df['Close']

# -------------------------------
# Hyperparameters
# -------------------------------
best_order = (2, 1, 3)
best_seasonal_order = (0, 1, 1, 22)

# -------------------------------
# Forecast settings
# -------------------------------
forecast_horizon = 5
step_size = 5
train_window_days = 500

forecast_dates = pd.date_range("2021-01-01", "2021-12-31", freq=f"{step_size}D")

# -------------------------------
# Forecasting function
# -------------------------------
def forecast_next_5_days(forecast_start):
    train_end = forecast_start - pd.Timedelta(days=1)
    train_start = train_end - pd.Timedelta(days=train_window_days - 1)

    if train_start < series.index[0]:
        return []

    train_data = series[train_start:train_end]

    try:
        model = SARIMAX(train_data,
                        order=best_order,
                        seasonal_order=best_seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        results = model.fit(disp=False, method_kwargs={"maxiter": 50})
        forecast = results.get_forecast(steps=forecast_horizon)
        preds = forecast.predicted_mean
    except:
        return []

    results_list = []
    for offset, pred_date in enumerate(pd.date_range(forecast_start, periods=forecast_horizon)):
        if pred_date not in series.index:
            continue
        results_list.append({
            "Date": pred_date,
            "Actual": series[pred_date],
            "Predicted": preds.iloc[offset]
        })
    return results_list

# -------------------------------
# Run forecasts in parallel
# -------------------------------
all_results = Parallel(n_jobs=-1)(
    delayed(forecast_next_5_days)(day) for day in tqdm(forecast_dates)
)

# Flatten list of lists
forecast_flat = [item for sublist in all_results for item in sublist]

# -------------------------------
# Evaluation & Plotting
# -------------------------------
forecast_df = pd.DataFrame(forecast_flat).dropna()
forecast_df.set_index("Date", inplace=True)

rmse = np.sqrt(mean_squared_error(forecast_df["Actual"], forecast_df["Predicted"]))
mae = mean_absolute_error(forecast_df["Actual"], forecast_df["Predicted"])
mape = mean_absolute_percentage_error(forecast_df["Actual"], forecast_df["Predicted"])
r2 = r2_score(forecast_df["Actual"], forecast_df["Predicted"])

print("\nSliding 5-Day Forecast Metrics:")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE:  {mae:.4f}")
print(f"  MAPE: {mape:.4f}")
print(f"  R2:   {r2:.4f}")

# Plot
plt.figure(figsize=(12, 6))
plt.plot(series["2021"], label="Actual")
plt.plot(forecast_df.index, forecast_df["Predicted"], label="5-day Forecasts", alpha=0.7)
plt.title("SARIMA: 5-Day-Ahead Forecasts Every 5 Days (2021)")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend()
plt.tight_layout()
plt.grid()
plt.show()

## 2.2 ETS

The code largely follows the same structure as SARIMA's. Below is the daily rolling-window prediction for AAPL 2024. This takes 1-2 minutes to run.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from joblib import Parallel, delayed
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# -------------------------------
# Load and preprocess data
# -------------------------------
df = pd.read_csv('AAPL.csv', skiprows=3, header=None)
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
series = df['Close']

# -------------------------------
# Forecast settings
# -------------------------------
forecast_horizon = 1
step_size = 1
train_window_days = 750

forecast_dates = series.loc["2024-01-01":"2024-12-31"].index[::step_size]

# -------------------------------
# Forecasting function
# -------------------------------
def forecast_next_5_days(forecast_start):
    train_end = forecast_start - pd.Timedelta(days=1)
    train_start = train_end - pd.Timedelta(days=train_window_days - 1)

    if train_start < series.index[0]:
        return []

    train_data = series[train_start:train_end]

    try:
        model = ExponentialSmoothing(train_data, trend='add', seasonal='mul',
                                     seasonal_periods=5, damped_trend=False)
        results = model.fit()
        forecast = results.forecast(steps=forecast_horizon)
    except Exception as e:
        print(f"Error forecasting for date {forecast_start}: {e}")
        return []

    results_list = []
    forecast_dates_range = pd.date_range(forecast_start, periods=forecast_horizon)
    for offset, pred_date in enumerate(forecast_dates_range):
        if pred_date in series.index and offset < len(forecast):
             results_list.append({
                "Date": pred_date,
                "Actual": series[pred_date],
                "Predicted": forecast.iloc[offset]
            })
    return results_list


# -------------------------------
# Run forecasts in parallel
# -------------------------------
all_results = Parallel(n_jobs=-1)(
    delayed(forecast_next_5_days)(day) for day in tqdm(forecast_dates)
)

# Flatten list of lists
forecast_flat = [item for sublist in all_results for item in sublist]

# -------------------------------
# Evaluation & Plotting
# -------------------------------
forecast_df = pd.DataFrame(forecast_flat).dropna()
forecast_df.set_index("Date", inplace=True)

rmse = np.sqrt(mean_squared_error(forecast_df["Actual"], forecast_df["Predicted"]))
mae = mean_absolute_error(forecast_df["Actual"], forecast_df["Predicted"])
mape = mean_absolute_percentage_error(forecast_df["Actual"], forecast_df["Predicted"])
r2 = r2_score(forecast_df["Actual"], forecast_df["Predicted"])

print("\nSliding 5-Day Forecast Metrics:")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE:  {mae:.4f}")
print(f"  MAPE: {mape:.4f}")
print(f"  R2:   {r2:.4f}")

# Plot
plt.figure(figsize=(12, 6))
plt.plot(series["2024"], label="Actual")
plt.plot(forecast_df.index, forecast_df["Predicted"], label="5-day Forecasts", alpha=0.7)
plt.title("ETS: Daily rolling-window Predictions, AAPL 2024")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend()
plt.tight_layout()
plt.show()

## 2.3 BSTS

This project only did full year predictions for BSTS. Unfortunately, the orbit package, necessary for this model, takes 20 minutes to build itself. The code is included nevertheless. Currently configured to predict JPM 2018.

In [ ]:
!pip install orbit-ml

In [ ]:
from orbit.models import DLT
from orbit.diagnostics.metrics import smape, mape
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

# -------------------------------
# Load and preprocess data
# -------------------------------
df = pd.read_csv('JPM.csv', skiprows=3, header=None)
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df = df[['Date', 'Close']]
df.columns = ['ds', 'y']
df = df.set_index('ds').asfreq('D')
df['y'] = df['y'].interpolate(method='time')
df = df.reset_index()

train_start = '2015-01-01'
train_end = '2017-12-31'
test_start = '2018-01-01'
test_end = '2018-12-31'

seasonality = 22

train_df = df[(df['ds'] >= train_start) & (df['ds'] <= train_end)].copy()
test_df = df[(df['ds'] >= test_start) & (df['ds'] <= test_end)].copy()

model = DLT(
    response_col='y',
    date_col='ds',
    seasonality=seasonality,
    seed=2023, # Reproducibility
    estimator='stan-mcmc',
    )
model.fit(train_df)

    # Forecast
forecast_df = model.predict(df=test_df)
y_true = test_df['y'].values
y_pred = forecast_df['prediction'].values

    # Evaluate
rmse_val = np.sqrt(mean_squared_error(y_true, y_pred))
mae_val = mean_absolute_error(y_true, y_pred)
mape_val = mape(y_true, y_pred)
smape_val = smape(y_true, y_pred)
r2_val = r2_score(y_true, y_pred)

print(f"Metrics:")
print(f"  RMSE:  {rmse_val:.4f}")
print(f"  MAE:   {mae_val:.4f}")
print(f"  MAPE:  {mape_val:.4f}")
print(f"  SMAPE: {smape_val:.4f}")
print(f"  R2:    {r2_val:.4f}")

# Plot
plt.figure(figsize=(10, 5))
plt.plot(train_df['ds'], train_df['y'], label='Train')
plt.plot(test_df['ds'], test_df['y'], label='Test')
plt.plot(forecast_df['ds'], forecast_df['prediction'], label='Forecast')
plt.fill_between(forecast_df['ds'], forecast_df['prediction_5'], forecast_df['prediction_95'], color='pink', alpha=0.3)
plt.title(f'Seasonality {seasonality} | BSTS Forecast (DLT model)')
plt.legend()
plt.tight_layout()
plt.show()

# 3. AI Models

## 3.1 LSTM

The ta package is used in the LSTM code. Installation takes 10 seconds.

In [ ]:
!pip install ta

This code predicts all 4 of the selected stocks (AAPL, JPM, NVDA, SPY) using a daily rolling-window. Currently configured to do this in 2024. This will run in 10-20 minutes on Colab's default CPU, or 5-6 minutes on the T4 GPU.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import warnings
from ta.momentum import RSIIndicator
from ta.trend import MACD
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from dateutil.relativedelta import relativedelta

warnings.filterwarnings("ignore")

def load_main_stock(path):
    df = pd.read_csv(path, skiprows=3, header=None)
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].set_index('Date')
    df = df.loc['2008-01-01':'2024-12-31']

    df['Close_lag1'] = df['Close'].shift(1)
    df['Volume_lag1'] = df['Volume'].shift(1)
    df['ma_5'] = df['Close_lag1'].rolling(window=5).mean()
    df['volatility_63'] = df['Close_lag1'].rolling(window=63).std()
    df['intraday_range'] = df['High'].shift(1) - df['Low'].shift(1)
    df['upper_shadow'] = df['High'].shift(1) - df[['Open', 'Close']].shift(1).max(axis=1)
    df['lower_shadow'] = df[['Open', 'Close']].shift(1).min(axis=1) - df['Low'].shift(1)
    df['momentum_126'] = df['Close_lag1'] - df['Close_lag1'].shift(126)
    df['RSI'] = RSIIndicator(close=df['Close_lag1'], window=14).rsi()
    macd = MACD(close=df['Close_lag1'])
    df['MACD'] = macd.macd()
    df['MACD_signal'] = macd.macd_signal()


    df = df.drop(columns=['Open', 'High', 'Low', 'Volume'])
    return df.dropna()

def create_sequences(X, y, window):
    Xs, ys = [], []
    for i in range(len(X) - window):
        Xs.append(X[i:i+window])
        ys.append(y[i+window])
    return np.array(Xs), np.array(ys)

def forecast_with_monthly_retraining(df, stock_name, window=20):
    df = df[['Close', 'Close_lag1']].dropna()
    feature_cols = df.columns.drop('Close')
    df = df.loc['2018-01-01':'2024-12-31']

    test_start = pd.Timestamp("2023-12-11")
    test_end = pd.Timestamp("2024-12-31")

    current_train_end = pd.Timestamp("2023-12-31")
    current_test_start = test_start

    all_preds = []
    all_true = []
    all_dates = []

    while current_test_start <= test_end:
        current_test_end = (current_test_start + relativedelta(months=1)) - pd.Timedelta(days=1)
        if current_test_end > test_end:
            current_test_end = test_end

        train_df = df.loc[:current_train_end]
        test_df = df.loc[current_test_start - pd.Timedelta(days=window):current_test_end]

        if len(test_df) <= window:
            break

        # Scaling
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_train = scaler_X.fit_transform(train_df[feature_cols])
        y_train = scaler_y.fit_transform(train_df[['Close']])

        X_test = scaler_X.transform(test_df[feature_cols])
        y_test = scaler_y.transform(test_df[['Close']])

        X_train_seq, y_train_seq = create_sequences(X_train, y_train, window)
        X_test_seq, y_test_seq = create_sequences(X_test, y_test, window)

        # Model
        model = Sequential([
            LSTM(64, input_shape=(window, X_train_seq.shape[2])),
            Dense(1)
        ])
        model.compile(optimizer=Adam(learning_rate=0.003), loss='mse')
        model.fit(X_train_seq, y_train_seq, epochs=20, batch_size=64, verbose=0)

        y_pred_scaled = model.predict(X_test_seq, verbose=0)
        y_pred = scaler_y.inverse_transform(y_pred_scaled)
        y_true = scaler_y.inverse_transform(y_test_seq)

        # Save results
        pred_dates = test_df.index[window:]
        all_preds.extend(y_pred.flatten())
        all_true.extend(y_true.flatten())
        all_dates.extend(pred_dates)

        # Append true values to training data for next iteration
        past_and_current = df.loc[:current_test_end]
        current_train_end = current_test_end + pd.Timedelta(days=0)
        current_test_start = current_test_end + pd.Timedelta(days=1)

    # Convert to pandas Series with datetime index
    pred_series = pd.Series(all_preds, index=pd.to_datetime(all_dates))
    true_series = pd.Series(all_true, index=pd.to_datetime(all_dates))

    # Truncate to only predictions from Jan 1, 2024
    plot_start = pd.Timestamp("2024-01-01")
    pred_series = pred_series[pred_series.index >= plot_start]
    true_series = true_series[true_series.index >= plot_start]

    # Use for metrics
    rmse = np.sqrt(mean_squared_error(true_series, pred_series))
    mae = mean_absolute_error(true_series, pred_series)
    r2 = r2_score(true_series, pred_series)
    mape = mean_absolute_percentage_error(true_series, pred_series)

    print(f"\n Results for {stock_name}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE : {mae:.4f}")
    print(f"R^2 : {r2:.4f}")
    print(f"MAPE: {mape:.4f}")

    # Convert to pandas Series for easy filtering
    pred_series = pd.Series(all_preds, index=pd.to_datetime(all_dates))
    true_series = pd.Series(all_true, index=pd.to_datetime(all_dates))

    # Filter to start from Jan 1, 2024
    plot_start = pd.Timestamp("2024-01-01")
    pred_series = pred_series[pred_series.index >= plot_start]
    true_series = true_series[true_series.index >= plot_start]

    plt.figure(figsize=(12, 6))
    plt.plot(true_series.index, true_series.values, label='Actual Close')
    plt.plot(pred_series.index, pred_series.values, label='Predicted Close')
    plt.title(f'{stock_name} - Monthly Retraining Forecast')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.legend()
    plt.grid()
    plt.show()

    scaled_rmse = rmse / (np.max(all_true) - np.min(all_true))
    return scaled_rmse

scaled_rmses = []

for ticker in ['AAPL.csv', 'JPM.csv', 'NVDA.csv', 'SPY.csv']:
    df = load_main_stock(ticker)
    scaled_rmse = forecast_with_monthly_retraining(df, ticker.replace('.csv', ''))
    scaled_rmses.append(scaled_rmse)

mean_scaled_rmse = sum(scaled_rmses) / len(scaled_rmses)
print(" Mean Scaled RMSE over 4 stocks:", mean_scaled_rmse)


## 3.2 TFT

This does not run in a reasonable time on the CPU. Change the runtime to the T4 GPU, if still using default CPU.

In [ ]:
!pip install pytorch_forecasting
!pip install torch
!pip install pytorch_lightning
!pip install ta

Like LSTM, this code also predicts AAPL, JPM, NVDA and SPY together, so they all need to be loaded. Currently configured for 2021. Takes 5-6 minutes on T4 GPU.

In [ ]:
import pandas as pd
import numpy as np
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics import SMAPE
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from collections import defaultdict
import warnings
import logging
from ta.momentum import RSIIndicator
from ta.trend import MACD

warnings.filterwarnings("ignore")
logging.getLogger("lightning.pytorch.accelerators.cuda").setLevel(logging.WARNING)

# === Load and preprocess data for all stocks ===
def load_stock(file_path, stock_name):
    df = pd.read_csv(file_path, skiprows=3, header=None)
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df['group'] = stock_name
    df['RSI'] = RSIIndicator(df['Close']).rsi()
    df['MACD'] = MACD(df['Close']).macd()
    df['upper_shadow'] = df['High'] - df[['Open', 'Close']].max(axis=1)
    df['lower_shadow'] = df[['Open', 'Close']].min(axis=1) - df['Low']
    return df

aapl = load_stock("AAPL.csv", "AAPL")
jpm  = load_stock("JPM.csv", "JPM")
nvda = load_stock("NVDA.csv", "NVDA")
spy  = load_stock("SPY.csv",  "SPY")

df = pd.concat([jpm, nvda, spy, aapl], ignore_index=True)
df = df[(df['Date'] > '2015-01-01') & (df['Date'] <= '2021-12-31')]
df = df.sort_values(['group', 'Date'])
df['time_idx'] = df.groupby('group').cumcount()

# Split train/val and test (based on calendar year)
train_val_df = df[df['Date'] < "2021-01-01"].copy()
test_df = df[df['Date'] >= "2020-11-16"].copy()

# Recalculate time_idx in test_df based on prior max index
last_time_idx = train_val_df.groupby("group")["time_idx"].max().reset_index()
test_df = test_df.merge(last_time_idx, on="group", suffixes=("", "_last"))
test_df["time_idx"] = test_df.groupby("group").cumcount() + test_df["time_idx_last"] + 1
test_df.drop(columns=["time_idx_last"], inplace=True)

# === Define dataset parameters ===
max_encoder_length = 30
max_prediction_length = 1
training_cutoff = train_val_df["time_idx"].max() - 252  # leave last 252 days for validation

# === Training dataset ===
tft_dataset = TimeSeriesDataSet(
    train_val_df[train_val_df.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Close",
    group_ids=["group"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["Close", "RSI", "MACD", 'upper_shadow', 'lower_shadow'],
    static_categoricals=["group"],
    allow_missing_timesteps=True
)

# === Validation dataset (pre-2024 only) ===
val_df = train_val_df[(train_val_df.time_idx > (training_cutoff - max_encoder_length))]
validation = TimeSeriesDataSet.from_dataset(tft_dataset, val_df, stop_randomization=True)

train_loader = tft_dataset.to_dataloader(train=True, batch_size=64, num_workers=0)
val_loader = validation.to_dataloader(train=False, batch_size=64, num_workers=0)

# === Define LightningModule wrapper ===
class TFTLightningModule(LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        y_pred = out[0]
        loss = self.model.loss(y_pred, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=x['encoder_target'].shape[0])
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        y_pred = out[0]
        loss = self.model.loss(y_pred, y)
        self.log("val_loss", loss, prog_bar=True, batch_size=x['encoder_target'].shape[0])
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.model.hparams.learning_rate)

# === Train TFT model ===
tft = TemporalFusionTransformer.from_dataset(
    tft_dataset,
    learning_rate=0.01,
    hidden_size=64,
    attention_head_size=2,
    dropout=0.1,
    loss=SMAPE(),
    log_interval=10,
    reduce_on_plateau_patience=4,
).to("cuda")

module = TFTLightningModule(tft)

trainer = Trainer(
    max_epochs=20,
    gradient_clip_val=0.1,
    callbacks=[EarlyStopping(monitor="val_loss", patience=5, mode="min")],
    limit_train_batches=30,
    logger=TensorBoardLogger("lightning_logs")
)

trainer.fit(module, train_dataloaders=train_loader, val_dataloaders=val_loader)


# === Rolling prediction on 2024 test data ===
preds_by_group = defaultdict(list)
actuals_by_group = defaultdict(list)

for group in test_df["group"].unique():
    df_group = test_df[test_df["group"] == group].copy()
    df_group = df_group.reset_index(drop=True)
    df_group["time_idx"] = np.arange(len(df_group))  # reindex cleanly

    steps = len(df_group["time_idx"].unique()) - max_encoder_length - max_prediction_length

    for offset in range(steps):
        decoder_end_time = offset + max_encoder_length + max_prediction_length - 1
        data_slice = df_group[df_group.time_idx <= decoder_end_time].tail(max_encoder_length + max_prediction_length)

        if data_slice['time_idx'].nunique() < (max_encoder_length + max_prediction_length):
            continue

        try:
            predict_dataset = TimeSeriesDataSet.from_dataset(tft_dataset, data_slice, stop_randomization=True)
            predict_loader = predict_dataset.to_dataloader(train=False, batch_size=1, num_workers=0)

            prediction = tft.predict(predict_loader).cpu().numpy().flatten()[0]
            actual = data_slice.iloc[-1]["Close"]

            preds_by_group[group].append(prediction)
            actuals_by_group[group].append(actual)
        except Exception as e:
            print(f"Skipping {group} at offset {offset} due to error: {e}")


# === Evaluation & Plotting ===
for group in preds_by_group:
    preds = preds_by_group[group]
    acts = actuals_by_group[group]

    if len(preds) == 0 or len(acts) == 0:
        print(f"\nNot enough data to evaluate {group}. Skipping...")
        continue

    rmse = np.sqrt(mean_squared_error(acts, preds))
    mae = mean_absolute_error(acts, preds)
    r2 = r2_score(acts, preds)
    mape = np.mean(np.abs((np.array(acts) - np.array(preds)) / np.array(acts))) * 100

    print(f"\n {group} Evaluation:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE : {mae:.4f}")
    print(f"  R²  : {r2:.4f}")
    print(f"  MAPE: {mape:.2f}%")

    plt.figure(figsize=(12, 5))
    plt.plot(acts, label=f"{group} Actual")
    plt.plot(preds, label=f"{group} Predicted")
    plt.title(f"{group} - 1-Day Ahead Rolling Forecast (2021)")
    plt.xlabel("Trading Days")
    plt.ylabel("Close Price")
    plt.legend()
    plt.grid()
    plt.show()

# 4. Graph Neural Networks

The 80 securities need to be loaded for graph construction. This takes 1-5 minutes. The current setup predicts SPY in 2021.

In [ ]:
!pip install torch-geometric ta

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch_geometric.data import Data
from functools import reduce
import joblib
from ta.momentum import RSIIndicator
from ta.trend import MACD

# Load and preprocess function (only scaled close price)
def load_and_scale_close(filepath):
    df = pd.read_csv(filepath, skiprows=3, header=None)
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df = df[['Date', 'Close']]
    df['RSI'] = RSIIndicator(df['Close']).rsi()
    df['MACD'] = MACD(df['Close']).macd()

    scaler_close = MinMaxScaler()
    scaler = MinMaxScaler()
    df['Close'] = scaler_close.fit_transform(df[['Close']])
    df['RSI'] = scaler.fit_transform(df[['RSI']])
    df['MACD'] = scaler.fit_transform(df[['MACD']])

    df = df[(df['Date'] >= '2015-01-01') & (df['Date'] <= '2021-12-31')]
    return df, scaler_close

selected_stocks = ['AAPL', 'JPM', 'NVDA', 'GOOGL', 'MSFT', 'AMZN', 'META', #7
                   'TSLA', 'SONY', 'JBL', 'NFLX',  'AMD', 'TSM', 'ORCL', 'AVGO',
                   'V', 'MS', 'COF', 'MET', 'AMAT', 'MCHP', 'ADI', #22
                   'BAC', 'BLK', "ASML", 'QCOM', 'CRM', 'ADBE', 'GS', 'WFC', #30
                   'SCHW', 'BK', 'AXP', 'TXN', 'MU', 'NXPI', 'KLAC', 'LRCX', #38
                   'UNH', 'JNJ', 'PFE', 'MRK', 'LLY', 'TMO', 'BMY', 'LMT', 'F',
                   'GM', 'HMC', 'TM', 'WMT', 'HD', 'COST', 'PG', 'KO', 'MCD',#56
                   'TGT', 'PEP', 'JNJ', 'NVO'] #60
# Related to the 4 i need + healthcare, automotive and consumer, for diversity.
selected_etfs = ['SPY', 'QQQ', 'DIA', 'IWM', 'XLK', 'XLF', 'XLE', 'XLI', 'XLV',
                 'XLY', 'XLP', 'VNQ', 'IYR', 'VGT', 'VTI', 'VUG', 'VTV', 'IWF',
                 'IWD', 'ITOT'] # 20.

target_stock = 'SPY'

panel = {}

# Load and scale AAPL separately to save its scaler
aapl_filepath = f"{target_stock}.csv"
aapl_df, aapl_scaler = load_and_scale_close(aapl_filepath)
panel[target_stock] = aapl_df
joblib.dump(aapl_scaler, f'scaler_{target_stock.lower()}.save')

# Load and scale other tickers
tickers_to_process = [t for t in selected_stocks + selected_etfs if t != target_stock]

for ticker in tickers_to_process:
    df, _ = load_and_scale_close(f"{ticker}.csv") # Scale each individually
    panel[ticker] = df


# Select only common dates
dataframes_to_merge = [df[['Date']] for df in panel.values()]
if not dataframes_to_merge:
    print("No dataframes loaded to find common dates.")
else:
    common_dates = reduce(lambda x, y: pd.merge(x, y, on='Date', how='inner'), dataframes_to_merge)
    common_dates = pd.to_datetime(common_dates['Date'].unique())
    common_dates = sorted(common_dates.tolist()) # Corrected sorting


    # Filter dataframes by common dates
    for ticker in panel:
        panel[ticker] = panel[ticker][panel[ticker]['Date'].isin(common_dates)].reset_index(drop=True)

    # Create graph list
    tickers_in_panel = list(panel.keys())
    target_indices = [tickers_in_panel.index(target_stock)]

    def fully_connected_edges(num_nodes):
        row = torch.arange(num_nodes).repeat_interleave(num_nodes)
        col = torch.arange(num_nodes).repeat(num_nodes)
        return torch.stack([row, col], dim=0)

    window_size = 21  # today + previous 20 days

    graph_list = []
    num_days = len(common_dates) - 1  # Leave 1 for next-day label

    for day in range(window_size - 1, num_days):
        x = []
        y = []

        for ticker in tickers_to_process:
            # Collect features from day-(window_size-1) to day (inclusive)
            window_rows = panel[ticker].iloc[day - (window_size - 1): day + 1]
            # Extract features for each day in window: Close, RSI, MACD
            # Shape: (window_size, 3)
            features_window = window_rows[['Close', 'RSI', 'MACD']].values.flatten()
            x.append(features_window)

        y.append(panel[target_stock].iloc[day + 1]['Close'])  # Next day close as label

        x_tensor = torch.tensor(x, dtype=torch.float)
        y_tensor = torch.tensor(y, dtype=torch.float)
        x_tensor = torch.nan_to_num(x_tensor, nan=0.0, posinf=0.0, neginf=0.0)
        y_tensor = torch.nan_to_num(y_tensor, nan=0.0, posinf=0.0, neginf=0.0)

        edge_index = fully_connected_edges(len(tickers_to_process))
        target_tensor = torch.tensor(target_indices, dtype=torch.long)

        data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor, target_node_ids=target_tensor)
        graph_list.append(data)

    torch.save(graph_list, 'graph_list_spy_21day_rsimacd21.pt')

With the saved graph list, we can run the GAT to get predictions. This takes up to 5 minutes on the T4 GPU. This code returns the prediction and the attention weights of the 10 most related stocks.

In [ ]:
# === Model Definition ===
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# === Load Graph Data ===
target_stock = 'SPY'
graph_list = torch.load(f"graph_list_{target_stock.lower()}_21day_rsimacd21.pt", weights_only=False)

test_days = 252
train_graphs = graph_list[:-test_days]
test_graphs = graph_list[-test_days:]

batch_size = 1
train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_graphs, batch_size=batch_size, shuffle=False)

# === Model Definition ===
class GATPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=4, dropout_prob=0.1):
        super(GATPredictor, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.fc = nn.Linear(hidden_channels * heads, out_channels)  # Predict price for each node
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x, (edge_index_attn, attn_weights_tensor) = self.gat1(
            x, edge_index, return_attention_weights=True
        )
        x = F.elu(x)
        x = self.dropout(x)  # Apply dropout after activation
        output = self.fc(x)  # shape: [num_nodes, 1]
        return output, (edge_index_attn, attn_weights_tensor)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GATPredictor(in_channels=63, hidden_channels=32, out_channels=1).to(device) # Changed in_channels to 1
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.MSELoss()

# === Training / Testing ===
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, _ = model(data)  # predictions for all nodes, shape [num_nodes, 1]
        out_target_nodes = out[data.target_node_ids].squeeze() # select and squeeze predictions for target nodes, shape [num_targets]
        # Ensure shapes are compatible for MSELoss, unsqueeze if scalar
        if out_target_nodes.ndim == 0:
            out_target_nodes = out_target_nodes.unsqueeze(0)
        if data.y.ndim == 0:
            data.y = data.y.unsqueeze(0)
        loss = loss_fn(out_target_nodes, data.y.to(device)) # compare with actual targets, shape [num_targets]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def test(loader):
    model.eval()
    total_loss = 0
    all_attn_weights = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out, (edge_index_attn, attn_weights_tensor) = model(data)  # predictions for all nodes, shape [num_nodes, 1]
            out_target_nodes = out[data.target_node_ids].squeeze() # select and squeeze predictions for target nodes, shape [num_targets]
            # Ensure shapes are compatible for MSELoss, unsqueeze if scalar
            if out_target_nodes.ndim == 0:
                out_target_nodes = out_target_nodes.unsqueeze(0)
            if data.y.ndim == 0:
                data.y = data.y.unsqueeze(0)
            loss = loss_fn(out_target_nodes, data.y.to(device)) # compare with actual targets, shape [num_targets]
            total_loss += loss.item()
            all_attn_weights.append((edge_index_attn.cpu(), attn_weights_tensor.cpu())) # Store edge_index and attention_weights_tensor

    return total_loss / len(loader), all_attn_weights

import copy

patience = 5   # how many epochs to wait for improvement
best_loss = float("inf")
epochs_no_improve = 0
best_model_state = None
n_epochs = 30  # or however many max you want

for epoch in range(n_epochs):
    train_loss = train()
    test_loss, test_attn_weights = test(test_loader)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early stopping check (based on train loss) ---
    if train_loss < best_loss - 1e-6:   # small delta to avoid floating point noise
        best_loss = train_loss
        best_model_state = copy.deepcopy(model.state_dict())
        epochs_no_improve = 0
        print(f" New best model saved (Train Loss: {best_loss:.4f})")
    else:
        epochs_no_improve += 1
        print(f" No improvement for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= patience:
        print(f"\n Early stopping at epoch {epoch+1}")
        break

# --- Restore best model before evaluation ---
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\n Restored model with best Train Loss = {best_loss:.4f}")

# === Predictions ===
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out, _ = model(data)  # predictions for all nodes
        out_target = out[data.target_node_ids].squeeze().cpu().numpy()
        # Ensure out_target is treated as an iterable even if it's a scalar
        if out_target.ndim == 0:
            out_target = out_target.reshape(1)
        predictions.extend(out_target)
        actuals.extend(data.y.cpu().numpy())

# === Inverse scaling and Metrics ===
aapl_scaler = joblib.load(f'scaler_{target_stock.lower()}.save')

predictions = aapl_scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
actuals = aapl_scaler.inverse_transform(np.array(actuals).reshape(-1, 1)).flatten()

rmse = np.sqrt(mean_squared_error(actuals, predictions))
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100
r2 = r2_score(actuals, predictions)

print(f"{target_stock} — RMSE: {rmse:.4f} | MAE: {mae:.4f} | MAPE: {mape:.2f}% | R²: {r2:.4f}")

# === Plotting ===
plt.figure(figsize=(10, 4))
plt.plot(actuals, label="Actual", color='black')
plt.plot(predictions, label="Predicted", color='red', alpha=0.7)
plt.title(f"{target_stock}: Actual vs Predicted Close Price (Over Time)")
plt.xlabel("Days")
plt.ylabel("Close Price")
plt.legend()
plt.grid(True)
plt.show()

# ==================
# Attention Weights
# ==================

# Extract attention weights for the target node
target_stock = 'SPY'

tickers_in_panel = list(panel.keys())
aapl_node_index = tickers_in_panel.index(target_stock)

aapl_attention_weights = []

for edge_index, attn_weights in test_attn_weights:
    # Find edges where the target node is AAPL
    # In a fully connected graph, the target node is in the second row of edge_index
    # and its index is aapl_node_index
    aapl_edges_mask = edge_index[1] == aapl_node_index

    # Get the attention weights for these edges
    weights_to_aapl = attn_weights[aapl_edges_mask]
    aapl_attention_weights.append(weights_to_aapl)

# aapl_attention_weights is a list of tensors, where each tensor contains the attention
# weights from all other nodes towards AAPL for a given time step in the test set.

print(f"Extracted attention weights for {target_stock} for {len(aapl_attention_weights)} time steps.")

if aapl_attention_weights:
    print(f"Shape of attention weights for one time step: {aapl_attention_weights[0].shape}")


# Calculate average attention weights across the test set
# Assuming aapl_attention_weights is a list of tensors where each tensor is [num_nodes, heads]
# We want to average across the time steps (the list) and potentially across heads

if aapl_attention_weights:
    # Concatenate all attention weight tensors from the list
    all_weights_tensor = torch.cat(aapl_attention_weights, dim=0) # Shape: [num_days * num_nodes, heads]

    # Reshape to [num_days, num_nodes, heads] and average over days
    num_days_test = len(aapl_attention_weights)
    num_nodes = all_weights_tensor.shape[0] // num_days_test
    num_heads = all_weights_tensor.shape[1]

    average_weights_per_node_head = all_weights_tensor.reshape(num_days_test, num_nodes, num_heads).mean(dim=0) # Shape: [num_nodes, heads]

    # Average across heads to get a single weight per node
    average_weights_per_node = average_weights_per_node_head.mean(dim=1) # Shape: [num_nodes]

    # Get the tickers corresponding to the nodes
    # Use tickers_to_process as the index, since the graph was built using these tickers

    # Create a pandas Series for easier sorting and visualization
    average_attention_series = pd.Series(average_weights_per_node.numpy(), index=tickers_in_panel) # Use corrected index

    # Sort the series by attention weight
    sorted_attention = average_attention_series.sort_values(ascending=False)

    # Plot the top N attention weights
    top_n = 10 # Adjustable top_n
    plt.figure(figsize=(12, 6))
    sorted_attention.head(top_n).plot(kind='bar')
    plt.title(f"Top {top_n} Average Attention Weights Towards {target_stock}")
    plt.xlabel("Ticker")
    plt.ylabel("Average Attention Weight")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    print(f"Top {top_n} tickers by average attention weight towards {target_stock}:")
    print(sorted_attention.head(top_n))

else:
    print("No attention weights were extracted.")

# 5. TFT GNN

To run all of the code necessary for the TFT-GNN to work can take up to 30 minutes on the T4 GPU. All 80 stocks must be loaded. GNN predictions are made for each of the target stocks: AAPL, JPM, NVDA, and SPY, and then fed into the TFT. The code is setup to predict 2024.

In [ ]:
!pip install pytorch_forecasting
!pip install torch
!pip install pytorch_lightning
!pip install torch-geometric ta

Graph construction for AAPL, JPM, NVDA, and SPY.

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch_geometric.data import Data
from functools import reduce
import joblib
from ta.momentum import RSIIndicator
from ta.trend import MACD

# Load and preprocess function (only scaled close price)
def load_and_scale_close(filepath):
    df = pd.read_csv(filepath, skiprows=3, header=None)
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df = df[['Date', 'Close']]
    df['RSI'] = RSIIndicator(df['Close']).rsi()
    df['MACD'] = MACD(df['Close']).macd()

    scaler_close = MinMaxScaler()
    scaler = MinMaxScaler()
    df['Close'] = scaler_close.fit_transform(df[['Close']])
    df['RSI'] = scaler.fit_transform(df[['RSI']])
    df['MACD'] = scaler.fit_transform(df[['MACD']])

    # Find the date 21 entries before '2018-01-01' based on available dates
    cutoff_date = pd.to_datetime('2018-01-01')
    dates_before_cutoff = df[df['Date'] < cutoff_date]['Date'].sort_values(ascending=False).tolist()
    earliest_date_needed = dates_before_cutoff[min(len(dates_before_cutoff) - 1, 20)] if dates_before_cutoff else cutoff_date


    df = df[(df['Date'] >= earliest_date_needed) & (df['Date'] <= '2024-12-31')]
    return df, scaler_close

selected_stocks = ['AAPL', 'JPM', 'NVDA', 'GOOGL', 'MSFT', 'AMZN', 'META', #7
                   'TSLA', 'SONY', 'JBL', 'NFLX',  'AMD', 'TSM', 'ORCL', 'AVGO',
                   'V', 'MS', 'COF', 'MET', 'AMAT', 'MCHP', 'ADI', #22
                   'BAC', 'BLK', "ASML", 'QCOM', 'CRM', 'ADBE', 'GS', 'WFC', #30
                   'SCHW', 'BK', 'AXP', 'TXN', 'MU', 'NXPI', 'KLAC', 'LRCX', #38
                   'UNH', 'JNJ', 'PFE', 'MRK', 'LLY', 'TMO', 'BMY', 'LMT', 'F',
                   'GM', 'HMC', 'TM', 'WMT', 'HD', 'COST', 'PG', 'KO', 'MCD',#56
                   'TGT', 'PEP', 'JNJ', 'NVO'] #60
# Related to the 4 i need + healthcare, automotive and consumer, for diversity.
selected_etfs = ['SPY', 'QQQ', 'DIA', 'IWM', 'XLK', 'XLF', 'XLE', 'XLI', 'XLV',
                 'XLY', 'XLP', 'VNQ', 'IYR', 'VGT', 'VTI', 'VUG', 'VTV', 'IWF',
                 'IWD', 'ITOT'] # 20.

target_stock = 'AAPL'

panel = {}

# Load and scale AAPL separately to save its scaler
aapl_filepath = f"{target_stock}.csv"
aapl_df, aapl_scaler = load_and_scale_close(aapl_filepath)
panel[target_stock] = aapl_df
joblib.dump(aapl_scaler, f'scaler_{target_stock.lower()}.save')

# Load and scale other tickers
tickers_to_process = [t for t in selected_stocks + selected_etfs if t != target_stock]

for ticker in tickers_to_process:
    df, _ = load_and_scale_close(f"{ticker}.csv") # Scale each individually
    panel[ticker] = df


# Select only common dates
dataframes_to_merge = [df[['Date']] for df in panel.values()]
if not dataframes_to_merge:
    print("No dataframes loaded to find common dates.")
else:
    common_dates = reduce(lambda x, y: pd.merge(x, y, on='Date', how='inner'), dataframes_to_merge)
    common_dates = pd.to_datetime(common_dates['Date'].unique())
    common_dates = sorted(common_dates.tolist()) # Corrected sorting


    # Filter dataframes by common dates
    for ticker in panel:
        panel[ticker] = panel[ticker][panel[ticker]['Date'].isin(common_dates)].reset_index(drop=True)

    # Create graph list
    tickers_in_panel = list(panel.keys())
    target_indices = [tickers_in_panel.index(target_stock)]

    def fully_connected_edges(num_nodes):
        row = torch.arange(num_nodes).repeat_interleave(num_nodes)
        col = torch.arange(num_nodes).repeat(num_nodes)
        return torch.stack([row, col], dim=0)

    window_size = 21  # today + previous 20 days

    graph_list = []
    num_days = len(common_dates) - 1  # Leave 1 for next-day label

    for day in range(window_size - 1, num_days):
        x = []
        y = []

        for ticker in tickers_to_process:
            # Collect features from day-(window_size-1) to day (inclusive)
            window_rows = panel[ticker].iloc[day - (window_size - 1): day + 1]
            # Extract features for each day in window: Close, RSI, MACD
            # Shape: (window_size, 3)
            features_window = window_rows[['Close']].values.flatten()
            x.append(features_window)

        y.append(panel[target_stock].iloc[day + 1]['Close'])  # Next day close as label

        x_tensor = torch.tensor(x, dtype=torch.float)
        y_tensor = torch.tensor(y, dtype=torch.float)
        x_tensor = torch.nan_to_num(x_tensor, nan=0.0, posinf=0.0, neginf=0.0)
        y_tensor = torch.nan_to_num(y_tensor, nan=0.0, posinf=0.0, neginf=0.0)

        edge_index = fully_connected_edges(len(tickers_to_process))
        target_tensor = torch.tensor(target_indices, dtype=torch.long)

        data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor, target_node_ids=target_tensor)
        graph_list.append(data)

    torch.save(graph_list, 'graph_list_aapl_21day_rsimacd.pt')

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch_geometric.data import Data
from functools import reduce
import joblib
from ta.momentum import RSIIndicator
from ta.trend import MACD

# Load and preprocess function (only scaled close price)
def load_and_scale_close(filepath):
    df = pd.read_csv(filepath, skiprows=3, header=None)
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df = df[['Date', 'Close']]
    df['RSI'] = RSIIndicator(df['Close']).rsi()
    df['MACD'] = MACD(df['Close']).macd()

    scaler_close = MinMaxScaler()
    scaler = MinMaxScaler()
    df['Close'] = scaler_close.fit_transform(df[['Close']])
    df['RSI'] = scaler.fit_transform(df[['RSI']])
    df['MACD'] = scaler.fit_transform(df[['MACD']])

    # Find the date 21 entries before '2018-01-01' based on available dates
    cutoff_date = pd.to_datetime('2018-01-01')
    dates_before_cutoff = df[df['Date'] < cutoff_date]['Date'].sort_values(ascending=False).tolist()
    earliest_date_needed = dates_before_cutoff[min(len(dates_before_cutoff) - 1, 20)] if dates_before_cutoff else cutoff_date


    df = df[(df['Date'] >= earliest_date_needed) & (df['Date'] <= '2024-12-31')]
    return df, scaler_close

selected_stocks = ['AAPL', 'JPM', 'NVDA', 'GOOGL', 'MSFT', 'AMZN', 'META', #7
                   'TSLA', 'SONY', 'JBL', 'NFLX',  'AMD', 'TSM', 'ORCL', 'AVGO',
                   'V', 'MS', 'COF', 'MET', 'AMAT', 'MCHP', 'ADI', #22
                   'BAC', 'BLK', "ASML", 'QCOM', 'CRM', 'ADBE', 'GS', 'WFC', #30
                   'SCHW', 'BK', 'AXP', 'TXN', 'MU', 'NXPI', 'KLAC', 'LRCX', #38
                   'UNH', 'JNJ', 'PFE', 'MRK', 'LLY', 'TMO', 'BMY', 'LMT', 'F',
                   'GM', 'HMC', 'TM', 'WMT', 'HD', 'COST', 'PG', 'KO', 'MCD',#56
                   'TGT', 'PEP', 'JNJ', 'NVO'] #60
# Related to the 4 i need + healthcare, automotive and consumer, for diversity.
selected_etfs = ['SPY', 'QQQ', 'DIA', 'IWM', 'XLK', 'XLF', 'XLE', 'XLI', 'XLV',
                 'XLY', 'XLP', 'VNQ', 'IYR', 'VGT', 'VTI', 'VUG', 'VTV', 'IWF',
                 'IWD', 'ITOT'] # 20.

target_stock = 'JPM'

panel = {}

# Load and scale AAPL separately to save its scaler
aapl_filepath = f"{target_stock}.csv"
aapl_df, aapl_scaler = load_and_scale_close(aapl_filepath)
panel[target_stock] = aapl_df
joblib.dump(aapl_scaler, f'scaler_{target_stock.lower()}.save')

# Load and scale other tickers
tickers_to_process = [t for t in selected_stocks + selected_etfs if t != target_stock]

for ticker in tickers_to_process:
    df, _ = load_and_scale_close(f"{ticker}.csv") # Scale each individually
    panel[ticker] = df


# Select only common dates
dataframes_to_merge = [df[['Date']] for df in panel.values()]
if not dataframes_to_merge:
    print("No dataframes loaded to find common dates.")
else:
    common_dates = reduce(lambda x, y: pd.merge(x, y, on='Date', how='inner'), dataframes_to_merge)
    common_dates = pd.to_datetime(common_dates['Date'].unique())
    common_dates = sorted(common_dates.tolist()) # Corrected sorting


    # Filter dataframes by common dates
    for ticker in panel:
        panel[ticker] = panel[ticker][panel[ticker]['Date'].isin(common_dates)].reset_index(drop=True)

    # Create graph list
    tickers_in_panel = list(panel.keys())
    target_indices = [tickers_in_panel.index(target_stock)]

    def fully_connected_edges(num_nodes):
        row = torch.arange(num_nodes).repeat_interleave(num_nodes)
        col = torch.arange(num_nodes).repeat(num_nodes)
        return torch.stack([row, col], dim=0)

    window_size = 21  # today + previous 20 days

    graph_list = []
    num_days = len(common_dates) - 1  # Leave 1 for next-day label

    for day in range(window_size - 1, num_days):
        x = []
        y = []

        for ticker in tickers_to_process:
            # Collect features from day-(window_size-1) to day (inclusive)
            window_rows = panel[ticker].iloc[day - (window_size - 1): day + 1]
            # Extract features for each day in window: Close, RSI, MACD
            # Shape: (window_size, 3)
            features_window = window_rows[['Close']].values.flatten()
            x.append(features_window)

        y.append(panel[target_stock].iloc[day + 1]['Close'])  # Next day close as label

        x_tensor = torch.tensor(x, dtype=torch.float)
        y_tensor = torch.tensor(y, dtype=torch.float)
        x_tensor = torch.nan_to_num(x_tensor, nan=0.0, posinf=0.0, neginf=0.0)
        y_tensor = torch.nan_to_num(y_tensor, nan=0.0, posinf=0.0, neginf=0.0)

        edge_index = fully_connected_edges(len(tickers_to_process))
        target_tensor = torch.tensor(target_indices, dtype=torch.long)

        data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor, target_node_ids=target_tensor)
        graph_list.append(data)

    torch.save(graph_list, 'graph_list_jpm_21day_rsimacd.pt')

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch_geometric.data import Data
from functools import reduce
import joblib
from ta.momentum import RSIIndicator
from ta.trend import MACD

# Load and preprocess function (only scaled close price)
def load_and_scale_close(filepath):
    df = pd.read_csv(filepath, skiprows=3, header=None)
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df = df[['Date', 'Close']]
    df['RSI'] = RSIIndicator(df['Close']).rsi()
    df['MACD'] = MACD(df['Close']).macd()

    scaler_close = MinMaxScaler()
    scaler = MinMaxScaler()
    df['Close'] = scaler_close.fit_transform(df[['Close']])
    df['RSI'] = scaler.fit_transform(df[['RSI']])
    df['MACD'] = scaler.fit_transform(df[['MACD']])

    # Find the date 21 entries before '2018-01-01' based on available dates
    cutoff_date = pd.to_datetime('2018-01-01')
    dates_before_cutoff = df[df['Date'] < cutoff_date]['Date'].sort_values(ascending=False).tolist()
    earliest_date_needed = dates_before_cutoff[min(len(dates_before_cutoff) - 1, 20)] if dates_before_cutoff else cutoff_date


    df = df[(df['Date'] >= earliest_date_needed) & (df['Date'] <= '2024-12-31')]
    return df, scaler_close

selected_stocks = ['AAPL', 'JPM', 'NVDA', 'GOOGL', 'MSFT', 'AMZN', 'META', #7
                   'TSLA', 'SONY', 'JBL', 'NFLX',  'AMD', 'TSM', 'ORCL', 'AVGO',
                   'V', 'MS', 'COF', 'MET', 'AMAT', 'MCHP', 'ADI', #22
                   'BAC', 'BLK', "ASML", 'QCOM', 'CRM', 'ADBE', 'GS', 'WFC', #30
                   'SCHW', 'BK', 'AXP', 'TXN', 'MU', 'NXPI', 'KLAC', 'LRCX', #38
                   'UNH', 'JNJ', 'PFE', 'MRK', 'LLY', 'TMO', 'BMY', 'LMT', 'F',
                   'GM', 'HMC', 'TM', 'WMT', 'HD', 'COST', 'PG', 'KO', 'MCD',#56
                   'TGT', 'PEP', 'JNJ', 'NVO'] #60
# Related to the 4 i need + healthcare, automotive and consumer, for diversity.
selected_etfs = ['SPY', 'QQQ', 'DIA', 'IWM', 'XLK', 'XLF', 'XLE', 'XLI', 'XLV',
                 'XLY', 'XLP', 'VNQ', 'IYR', 'VGT', 'VTI', 'VUG', 'VTV', 'IWF',
                 'IWD', 'ITOT'] # 20.

target_stock = 'NVDA'

panel = {}

# Load and scale AAPL separately to save its scaler
aapl_filepath = f"{target_stock}.csv"
aapl_df, aapl_scaler = load_and_scale_close(aapl_filepath)
panel[target_stock] = aapl_df
joblib.dump(aapl_scaler, f'scaler_{target_stock.lower()}.save')

# Load and scale other tickers
tickers_to_process = [t for t in selected_stocks + selected_etfs if t != target_stock]

for ticker in tickers_to_process:
    df, _ = load_and_scale_close(f"{ticker}.csv") # Scale each individually
    panel[ticker] = df


# Select only common dates
dataframes_to_merge = [df[['Date']] for df in panel.values()]
if not dataframes_to_merge:
    print("No dataframes loaded to find common dates.")
else:
    common_dates = reduce(lambda x, y: pd.merge(x, y, on='Date', how='inner'), dataframes_to_merge)
    common_dates = pd.to_datetime(common_dates['Date'].unique())
    common_dates = sorted(common_dates.tolist()) # Corrected sorting


    # Filter dataframes by common dates
    for ticker in panel:
        panel[ticker] = panel[ticker][panel[ticker]['Date'].isin(common_dates)].reset_index(drop=True)

    # Create graph list
    tickers_in_panel = list(panel.keys())
    target_indices = [tickers_in_panel.index(target_stock)]

    def fully_connected_edges(num_nodes):
        row = torch.arange(num_nodes).repeat_interleave(num_nodes)
        col = torch.arange(num_nodes).repeat(num_nodes)
        return torch.stack([row, col], dim=0)

    window_size = 21  # today + previous 20 days

    graph_list = []
    num_days = len(common_dates) - 1  # Leave 1 for next-day label

    for day in range(window_size - 1, num_days):
        x = []
        y = []

        for ticker in tickers_to_process:
            # Collect features from day-(window_size-1) to day (inclusive)
            window_rows = panel[ticker].iloc[day - (window_size - 1): day + 1]
            # Extract features for each day in window: Close, RSI, MACD
            # Shape: (window_size, 3)
            features_window = window_rows[['Close']].values.flatten()
            x.append(features_window)

        y.append(panel[target_stock].iloc[day + 1]['Close'])  # Next day close as label

        x_tensor = torch.tensor(x, dtype=torch.float)
        y_tensor = torch.tensor(y, dtype=torch.float)
        x_tensor = torch.nan_to_num(x_tensor, nan=0.0, posinf=0.0, neginf=0.0)
        y_tensor = torch.nan_to_num(y_tensor, nan=0.0, posinf=0.0, neginf=0.0)

        edge_index = fully_connected_edges(len(tickers_to_process))
        target_tensor = torch.tensor(target_indices, dtype=torch.long)

        data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor, target_node_ids=target_tensor)
        graph_list.append(data)

    torch.save(graph_list, 'graph_list_nvda_21day_rsimacd.pt')

In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch_geometric.data import Data
from functools import reduce
import joblib
from ta.momentum import RSIIndicator
from ta.trend import MACD

# Load and preprocess function (only scaled close price)
def load_and_scale_close(filepath):
    df = pd.read_csv(filepath, skiprows=3, header=None)
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df = df[['Date', 'Close']]
    df['RSI'] = RSIIndicator(df['Close']).rsi()
    df['MACD'] = MACD(df['Close']).macd()

    scaler_close = MinMaxScaler()
    scaler = MinMaxScaler()
    df['Close'] = scaler_close.fit_transform(df[['Close']])
    df['RSI'] = scaler.fit_transform(df[['RSI']])
    df['MACD'] = scaler.fit_transform(df[['MACD']])

    # Find the date 21 entries before '2018-01-01' based on available dates
    cutoff_date = pd.to_datetime('2018-01-01')
    dates_before_cutoff = df[df['Date'] < cutoff_date]['Date'].sort_values(ascending=False).tolist()
    earliest_date_needed = dates_before_cutoff[min(len(dates_before_cutoff) - 1, 20)] if dates_before_cutoff else cutoff_date

    df = df[(df['Date'] >= earliest_date_needed) & (df['Date'] <= '2024-12-31')]
    return df, scaler_close

selected_stocks = ['AAPL', 'JPM', 'NVDA', 'GOOGL', 'MSFT', 'AMZN', 'META', #7
                   'TSLA', 'SONY', 'JBL', 'NFLX',  'AMD', 'TSM', 'ORCL', 'AVGO',
                   'V', 'MS', 'COF', 'MET', 'AMAT', 'MCHP', 'ADI', #22
                   'BAC', 'BLK', "ASML", 'QCOM', 'CRM', 'ADBE', 'GS', 'WFC', #30
                   'SCHW', 'BK', 'AXP', 'TXN', 'MU', 'NXPI', 'KLAC', 'LRCX', #38
                   'UNH', 'JNJ', 'PFE', 'MRK', 'LLY', 'TMO', 'BMY', 'LMT', 'F',
                   'GM', 'HMC', 'TM', 'WMT', 'HD', 'COST', 'PG', 'KO', 'MCD',#56
                   'TGT', 'PEP', 'JNJ', 'NVO'] #60
# Related to the 4 i need + healthcare, automotive and consumer, for diversity.
selected_etfs = ['SPY', 'QQQ', 'DIA', 'IWM', 'XLK', 'XLF', 'XLE', 'XLI', 'XLV',
                 'XLY', 'XLP', 'VNQ', 'IYR', 'VGT', 'VTI', 'VUG', 'VTV', 'IWF',
                 'IWD', 'ITOT'] # 20.

target_stock = 'SPY'

panel = {}

# Load and scale AAPL separately to save its scaler
aapl_filepath = f"{target_stock}.csv"
aapl_df, aapl_scaler = load_and_scale_close(aapl_filepath)
panel[target_stock] = aapl_df
joblib.dump(aapl_scaler, f'scaler_{target_stock.lower()}.save')

# Load and scale other tickers
tickers_to_process = [t for t in selected_stocks + selected_etfs if t != target_stock]

for ticker in tickers_to_process:
    df, _ = load_and_scale_close(f"{ticker}.csv") # Scale each individually
    panel[ticker] = df


# Select only common dates
dataframes_to_merge = [df[['Date']] for df in panel.values()]
if not dataframes_to_merge:
    print("No dataframes loaded to find common dates.")
else:
    common_dates = reduce(lambda x, y: pd.merge(x, y, on='Date', how='inner'), dataframes_to_merge)
    common_dates = pd.to_datetime(common_dates['Date'].unique())
    common_dates = sorted(common_dates.tolist()) # Corrected sorting


    # Filter dataframes by common dates
    for ticker in panel:
        panel[ticker] = panel[ticker][panel[ticker]['Date'].isin(common_dates)].reset_index(drop=True)

    # Create graph list
    tickers_in_panel = list(panel.keys())
    target_indices = [tickers_in_panel.index(target_stock)]

    def fully_connected_edges(num_nodes):
        row = torch.arange(num_nodes).repeat_interleave(num_nodes)
        col = torch.arange(num_nodes).repeat(num_nodes)
        return torch.stack([row, col], dim=0)

    window_size = 21  # today + previous 20 days

    graph_list = []
    num_days = len(common_dates) - 1  # Leave 1 for next-day label

    for day in range(window_size - 1, num_days):
        x = []
        y = []

        for ticker in tickers_to_process:
            # Collect features from day-(window_size-1) to day (inclusive)
            window_rows = panel[ticker].iloc[day - (window_size - 1): day + 1]
            # Extract features for each day in window: Close, RSI, MACD
            # Shape: (window_size, 3)
            features_window = window_rows[['Close']].values.flatten()
            x.append(features_window)

        y.append(panel[target_stock].iloc[day + 1]['Close'])  # Next day close as label

        x_tensor = torch.tensor(x, dtype=torch.float)
        y_tensor = torch.tensor(y, dtype=torch.float)
        x_tensor = torch.nan_to_num(x_tensor, nan=0.0, posinf=0.0, neginf=0.0)
        y_tensor = torch.nan_to_num(y_tensor, nan=0.0, posinf=0.0, neginf=0.0)

        edge_index = fully_connected_edges(len(tickers_to_process))
        target_tensor = torch.tensor(target_indices, dtype=torch.long)

        data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor, target_node_ids=target_tensor)
        graph_list.append(data)

    torch.save(graph_list, 'graph_list_spy_21day_rsimacd.pt')

Now using these 4 graph lists, we make GNN predictions.

In [ ]:
# === Model Definition ===
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# === Load Graph Data ===
target_stock = 'AAPL'
graph_list = torch.load(f"graph_list_{target_stock.lower()}_21day_rsimacd.pt", weights_only=False)

test_days = 252
train_graphs = graph_list[:-test_days]
test_graphs = graph_list

batch_size = 1
train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_graphs, batch_size=batch_size, shuffle=False)

# === Model Definition ===
class GATPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=4, dropout_prob=0.2):
        super(GATPredictor, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.fc = nn.Linear(hidden_channels * heads, out_channels)  # Predict price for each node
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x, (edge_index_attn, attn_weights_tensor) = self.gat1(
            x, edge_index, return_attention_weights=True
        )
        x = F.elu(x)
        x = self.dropout(x)  # Apply dropout after activation
        output = self.fc(x)  # shape: [num_nodes, 1]
        return output, (edge_index_attn, attn_weights_tensor)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GATPredictor(in_channels=21, hidden_channels=32, out_channels=1).to(device) # Changed in_channels to 1
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
loss_fn = nn.MSELoss()

# === Training / Testing ===
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, _ = model(data)  # predictions for all nodes, shape [num_nodes, 1]
        out_target_nodes = out[data.target_node_ids].squeeze() # select and squeeze predictions for target nodes, shape [num_targets]
        # Ensure shapes are compatible for MSELoss, unsqueeze if scalar
        if out_target_nodes.ndim == 0:
            out_target_nodes = out_target_nodes.unsqueeze(0)
        if data.y.ndim == 0:
            data.y = data.y.unsqueeze(0)
        loss = loss_fn(out_target_nodes, data.y.to(device)) # compare with actual targets, shape [num_targets]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def test(loader):
    model.eval()
    total_loss = 0
    all_attn_weights = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out, (edge_index_attn, attn_weights_tensor) = model(data)  # predictions for all nodes, shape [num_nodes, 1]
            out_target_nodes = out[data.target_node_ids].squeeze() # select and squeeze predictions for target nodes, shape [num_targets]
            # Ensure shapes are compatible for MSELoss, unsqueeze if scalar
            if out_target_nodes.ndim == 0:
                out_target_nodes = out_target_nodes.unsqueeze(0)
            if data.y.ndim == 0:
                data.y = data.y.unsqueeze(0)
            loss = loss_fn(out_target_nodes, data.y.to(device)) # compare with actual targets, shape [num_targets]
            total_loss += loss.item()
            all_attn_weights.append((edge_index_attn.cpu(), attn_weights_tensor.cpu())) # Store edge_index and attention_weights_tensor

    return total_loss / len(loader), all_attn_weights

import copy

patience = 5   # how many epochs to wait for improvement
best_loss = float("inf")
epochs_no_improve = 0
best_model_state = None
n_epochs = 30  # or however many max you want

for epoch in range(n_epochs):
    train_loss = train()
    test_loss, test_attn_weights = test(test_loader)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early stopping check (based on train loss) ---
    if test_loss < best_loss - 1e-6:   # small delta to avoid floating point noise
        best_loss = test_loss
        best_model_state = copy.deepcopy(model.state_dict())
        epochs_no_improve = 0
        print(f" New best model saved (Train Loss: {best_loss:.4f})")
    else:
        epochs_no_improve += 1
        print(f" No improvement for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= patience:
        print(f"\n Early stopping at epoch {epoch+1}")
        break

# --- Restore best model before evaluation ---
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\n Restored model with best Train Loss = {best_loss:.4f}")

# === Predictions ===
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out, _ = model(data)  # predictions for all nodes
        out_target = out[data.target_node_ids].squeeze().cpu().numpy()
        # Ensure out_target is treated as an iterable even if it's a scalar
        if out_target.ndim == 0:
            out_target = out_target.reshape(1)
        predictions.extend(out_target)
        actuals.extend(data.y.cpu().numpy())

# === Inverse scaling and Metrics ===
aapl_scaler = joblib.load(f'scaler_{target_stock.lower()}.save')

predictions = aapl_scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
actuals = aapl_scaler.inverse_transform(np.array(actuals).reshape(-1, 1)).flatten()

rmse = np.sqrt(mean_squared_error(actuals, predictions))
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100
r2 = r2_score(actuals, predictions)

print(f"{target_stock} — RMSE: {rmse:.4f} | MAE: {mae:.4f} | MAPE: {mape:.2f}% | R²: {r2:.4f}")

# === Plotting ===
plt.figure(figsize=(10, 4))
plt.plot(actuals, label="Actual", color='black')
plt.plot(predictions, label="Predicted", color='red', alpha=0.7)
plt.title(f"{target_stock}: Actual vs Predicted Close Price (Over Time)")
plt.xlabel("Days")
plt.ylabel("Close Price")
plt.legend()
plt.grid(True)
plt.show()

gnn_predictions = predictions

# assume gnn_predictions is a numpy array (len = N)
gnn_predictions = np.array(gnn_predictions)

# Compare today vs yesterday
gnn_updown = np.zeros_like(gnn_predictions, dtype=int)

for i in range(1, len(gnn_predictions)):
    if gnn_predictions[i] > gnn_predictions[i-1]:
        gnn_updown[i] = 1  # up
    else:
        gnn_updown[i] = 0  # down

# Force the first value to be "up" as you requested
gnn_updown[0] = 1
gnn_aapl_updown = gnn_updown



# Extract attention weights for the target node (AAPL)
target_stock = 'AAPL'
# Assuming 'tickers' list and 'panel' dictionary are available from previous cells
tickers_in_panel = list(panel.keys())
aapl_node_index = tickers_in_panel.index(target_stock)

aapl_attention_weights = []

for edge_index, attn_weights in test_attn_weights:
    # Find edges where the target node is AAPL
    # In a fully connected graph, the target node is in the second row of edge_index
    # and its index is aapl_node_index
    aapl_edges_mask = edge_index[1] == aapl_node_index

    # Get the attention weights for these edges
    weights_to_aapl = attn_weights[aapl_edges_mask]
    aapl_attention_weights.append(weights_to_aapl)

# aapl_attention_weights is a list of tensors, where each tensor contains the attention
# weights from all other nodes towards AAPL for a given time step in the test set.

print(f"Extracted attention weights for {target_stock} for {len(aapl_attention_weights)} time steps.")
# You can inspect the shape of the weights for a single time step
if aapl_attention_weights:
    print(f"Shape of attention weights for one time step: {aapl_attention_weights[0].shape}")


# Calculate average attention weights across the test set
# Assuming aapl_attention_weights is a list of tensors where each tensor is [num_nodes, heads]
# We want to average across the time steps (the list) and potentially across heads

if aapl_attention_weights:
    # Concatenate all attention weight tensors from the list
    all_weights_tensor = torch.cat(aapl_attention_weights, dim=0) # Shape: [num_days * num_nodes, heads]

    # Reshape to [num_days, num_nodes, heads] and average over days
    num_days_test = len(aapl_attention_weights)
    num_nodes = all_weights_tensor.shape[0] // num_days_test
    num_heads = all_weights_tensor.shape[1]

    average_weights_per_node_head = all_weights_tensor.reshape(num_days_test, num_nodes, num_heads).mean(dim=0) # Shape: [num_nodes, heads]

    # Average across heads to get a single weight per node
    average_weights_per_node = average_weights_per_node_head.mean(dim=1) # Shape: [num_nodes]

    # Get the tickers corresponding to the nodes
    # Use tickers_to_process as the index, since the graph was built using these tickers

    # Create a pandas Series for easier sorting and visualization
    average_attention_series = pd.Series(average_weights_per_node.numpy(), index=tickers_in_panel) # Use corrected index

    # Sort the series by attention weight
    sorted_attention = average_attention_series.sort_values(ascending=False)

    # Plot the top N attention weights
    top_n = 10 # Adjusted top_n
    plt.figure(figsize=(12, 6))
    sorted_attention.head(top_n).plot(kind='bar')
    plt.title(f"Top {top_n} Average Attention Weights Towards {target_stock}")
    plt.xlabel("Ticker")
    plt.ylabel("Average Attention Weight")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    print(f"Top {top_n} tickers by average attention weight towards {target_stock}:")
    print(sorted_attention.head(top_n))

else:
    print("No attention weights were extracted.")

In [ ]:
# === Model Definition ===
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# === Load Graph Data ===
target_stock = 'JPM'
graph_list = torch.load(f"graph_list_{target_stock.lower()}_21day_rsimacd.pt", weights_only=False)

test_days = 252
train_graphs = graph_list[:-test_days]
test_graphs = graph_list

batch_size = 1
train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_graphs, batch_size=batch_size, shuffle=False)

# === Model Definition ===
class GATPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=4, dropout_prob=0.2):
        super(GATPredictor, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.fc = nn.Linear(hidden_channels * heads, out_channels)  # Predict price for each node
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x, (edge_index_attn, attn_weights_tensor) = self.gat1(
            x, edge_index, return_attention_weights=True
        )
        x = F.elu(x)
        x = self.dropout(x)  # Apply dropout after activation
        output = self.fc(x)  # shape: [num_nodes, 1]
        return output, (edge_index_attn, attn_weights_tensor)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GATPredictor(in_channels=21, hidden_channels=32, out_channels=1).to(device) # Changed in_channels to 1
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
loss_fn = nn.MSELoss()

# === Training / Testing ===
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, _ = model(data)  # predictions for all nodes, shape [num_nodes, 1]
        out_target_nodes = out[data.target_node_ids].squeeze() # select and squeeze predictions for target nodes, shape [num_targets]
        # Ensure shapes are compatible for MSELoss, unsqueeze if scalar
        if out_target_nodes.ndim == 0:
            out_target_nodes = out_target_nodes.unsqueeze(0)
        if data.y.ndim == 0:
            data.y = data.y.unsqueeze(0)
        loss = loss_fn(out_target_nodes, data.y.to(device)) # compare with actual targets, shape [num_targets]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def test(loader):
    model.eval()
    total_loss = 0
    all_attn_weights = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out, (edge_index_attn, attn_weights_tensor) = model(data)  # predictions for all nodes, shape [num_nodes, 1]
            out_target_nodes = out[data.target_node_ids].squeeze() # select and squeeze predictions for target nodes, shape [num_targets]
            # Ensure shapes are compatible for MSELoss, unsqueeze if scalar
            if out_target_nodes.ndim == 0:
                out_target_nodes = out_target_nodes.unsqueeze(0)
            if data.y.ndim == 0:
                data.y = data.y.unsqueeze(0)
            loss = loss_fn(out_target_nodes, data.y.to(device)) # compare with actual targets, shape [num_targets]
            total_loss += loss.item()
            all_attn_weights.append((edge_index_attn.cpu(), attn_weights_tensor.cpu())) # Store edge_index and attention_weights_tensor

    return total_loss / len(loader), all_attn_weights

import copy

patience = 5   # how many epochs to wait for improvement
best_loss = float("inf")
epochs_no_improve = 0
best_model_state = None
n_epochs = 30  # or however many max you want

for epoch in range(n_epochs):
    train_loss = train()
    test_loss, test_attn_weights = test(test_loader)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early stopping check (based on train loss) ---
    if test_loss < best_loss - 1e-6:   # small delta to avoid floating point noise
        best_loss = test_loss
        best_model_state = copy.deepcopy(model.state_dict())
        epochs_no_improve = 0
        print(f" New best model saved (Train Loss: {best_loss:.4f})")
    else:
        epochs_no_improve += 1
        print(f" No improvement for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= patience:
        print(f"\n Early stopping at epoch {epoch+1}")
        break

# --- Restore best model before evaluation ---
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\n Restored model with best Train Loss = {best_loss:.4f}")

# === Predictions ===
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out, _ = model(data)  # predictions for all nodes
        out_target = out[data.target_node_ids].squeeze().cpu().numpy()
        # Ensure out_target is treated as an iterable even if it's a scalar
        if out_target.ndim == 0:
            out_target = out_target.reshape(1)
        predictions.extend(out_target)
        actuals.extend(data.y.cpu().numpy())

# === Inverse scaling and Metrics ===
aapl_scaler = joblib.load(f'scaler_{target_stock.lower()}.save')

predictions = aapl_scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
actuals = aapl_scaler.inverse_transform(np.array(actuals).reshape(-1, 1)).flatten()

rmse = np.sqrt(mean_squared_error(actuals, predictions))
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100
r2 = r2_score(actuals, predictions)

print(f"{target_stock} — RMSE: {rmse:.4f} | MAE: {mae:.4f} | MAPE: {mape:.2f}% | R²: {r2:.4f}")

# === Plotting ===
plt.figure(figsize=(10, 4))
plt.plot(actuals, label="Actual", color='black')
plt.plot(predictions, label="Predicted", color='red', alpha=0.7)
plt.title(f"{target_stock}: Actual vs Predicted Close Price (Over Time)")
plt.xlabel("Days")
plt.ylabel("Close Price")
plt.legend()
plt.grid(True)
plt.show()

gnn_predictions = predictions

# assume gnn_predictions is a numpy array (len = N)
gnn_predictions = np.array(gnn_predictions)

# Compare today vs yesterday
gnn_updown = np.zeros_like(gnn_predictions, dtype=int)

for i in range(1, len(gnn_predictions)):
    if gnn_predictions[i] > gnn_predictions[i-1]:
        gnn_updown[i] = 1  # up
    else:
        gnn_updown[i] = 0  # down

# Force the first value to be "up" as you requested
gnn_updown[0] = 1
gnn_jpm_updown = gnn_updown



# Extract attention weights for the target node (AAPL)
target_stock = 'JPM'
# Assuming 'tickers' list and 'panel' dictionary are available from previous cells
tickers_in_panel = list(panel.keys())
aapl_node_index = tickers_in_panel.index(target_stock)

aapl_attention_weights = []

for edge_index, attn_weights in test_attn_weights:
    # Find edges where the target node is AAPL
    # In a fully connected graph, the target node is in the second row of edge_index
    # and its index is aapl_node_index
    aapl_edges_mask = edge_index[1] == aapl_node_index

    # Get the attention weights for these edges
    weights_to_aapl = attn_weights[aapl_edges_mask]
    aapl_attention_weights.append(weights_to_aapl)

# aapl_attention_weights is a list of tensors, where each tensor contains the attention
# weights from all other nodes towards AAPL for a given time step in the test set.

print(f"Extracted attention weights for {target_stock} for {len(aapl_attention_weights)} time steps.")
# You can inspect the shape of the weights for a single time step
if aapl_attention_weights:
    print(f"Shape of attention weights for one time step: {aapl_attention_weights[0].shape}")


# Calculate average attention weights across the test set
# Assuming aapl_attention_weights is a list of tensors where each tensor is [num_nodes, heads]
# We want to average across the time steps (the list) and potentially across heads

if aapl_attention_weights:
    # Concatenate all attention weight tensors from the list
    all_weights_tensor = torch.cat(aapl_attention_weights, dim=0) # Shape: [num_days * num_nodes, heads]

    # Reshape to [num_days, num_nodes, heads] and average over days
    num_days_test = len(aapl_attention_weights)
    num_nodes = all_weights_tensor.shape[0] // num_days_test
    num_heads = all_weights_tensor.shape[1]

    average_weights_per_node_head = all_weights_tensor.reshape(num_days_test, num_nodes, num_heads).mean(dim=0) # Shape: [num_nodes, heads]

    # Average across heads to get a single weight per node
    average_weights_per_node = average_weights_per_node_head.mean(dim=1) # Shape: [num_nodes]

    # Get the tickers corresponding to the nodes
    # Use tickers_to_process as the index, since the graph was built using these tickers

    # Create a pandas Series for easier sorting and visualization
    average_attention_series = pd.Series(average_weights_per_node.numpy(), index=tickers_in_panel) # Use corrected index

    # Sort the series by attention weight
    sorted_attention = average_attention_series.sort_values(ascending=False)

    # Plot the top N attention weights
    top_n = 10 # Adjusted top_n
    plt.figure(figsize=(12, 6))
    sorted_attention.head(top_n).plot(kind='bar')
    plt.title(f"Top {top_n} Average Attention Weights Towards {target_stock}")
    plt.xlabel("Ticker")
    plt.ylabel("Average Attention Weight")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    print(f"Top {top_n} tickers by average attention weight towards {target_stock}:")
    print(sorted_attention.head(top_n))

else:
    print("No attention weights were extracted.")

In [ ]:
# === Model Definition ===
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# === Load Graph Data ===
target_stock = 'NVDA'
graph_list = torch.load(f"graph_list_{target_stock.lower()}_21day_rsimacd.pt", weights_only=False)

test_days = 252
train_graphs = graph_list[:-test_days]
test_graphs = graph_list

batch_size = 1
train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_graphs, batch_size=batch_size, shuffle=False)

# === Model Definition ===
class GATPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=4, dropout_prob=0.2):
        super(GATPredictor, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.fc = nn.Linear(hidden_channels * heads, out_channels)  # Predict price for each node
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x, (edge_index_attn, attn_weights_tensor) = self.gat1(
            x, edge_index, return_attention_weights=True
        )
        x = F.elu(x)
        x = self.dropout(x)  # Apply dropout after activation
        output = self.fc(x)  # shape: [num_nodes, 1]
        return output, (edge_index_attn, attn_weights_tensor)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GATPredictor(in_channels=21, hidden_channels=32, out_channels=1).to(device) # Changed in_channels to 1
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
loss_fn = nn.MSELoss()

# === Training / Testing ===
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, _ = model(data)  # predictions for all nodes, shape [num_nodes, 1]
        out_target_nodes = out[data.target_node_ids].squeeze() # select and squeeze predictions for target nodes, shape [num_targets]
        # Ensure shapes are compatible for MSELoss, unsqueeze if scalar
        if out_target_nodes.ndim == 0:
            out_target_nodes = out_target_nodes.unsqueeze(0)
        if data.y.ndim == 0:
            data.y = data.y.unsqueeze(0)
        loss = loss_fn(out_target_nodes, data.y.to(device)) # compare with actual targets, shape [num_targets]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def test(loader):
    model.eval()
    total_loss = 0
    all_attn_weights = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out, (edge_index_attn, attn_weights_tensor) = model(data)  # predictions for all nodes, shape [num_nodes, 1]
            out_target_nodes = out[data.target_node_ids].squeeze() # select and squeeze predictions for target nodes, shape [num_targets]
            # Ensure shapes are compatible for MSELoss, unsqueeze if scalar
            if out_target_nodes.ndim == 0:
                out_target_nodes = out_target_nodes.unsqueeze(0)
            if data.y.ndim == 0:
                data.y = data.y.unsqueeze(0)
            loss = loss_fn(out_target_nodes, data.y.to(device)) # compare with actual targets, shape [num_targets]
            total_loss += loss.item()
            all_attn_weights.append((edge_index_attn.cpu(), attn_weights_tensor.cpu())) # Store edge_index and attention_weights_tensor

    return total_loss / len(loader), all_attn_weights

import copy

patience = 5   # how many epochs to wait for improvement
best_loss = float("inf")
epochs_no_improve = 0
best_model_state = None
n_epochs = 30  # or however many max you want

for epoch in range(n_epochs):
    train_loss = train()
    test_loss, test_attn_weights = test(test_loader)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early stopping check (based on train loss) ---
    if test_loss < best_loss - 1e-6:   # small delta to avoid floating point noise
        best_loss = test_loss
        best_model_state = copy.deepcopy(model.state_dict())
        epochs_no_improve = 0
        print(f" New best model saved (Train Loss: {best_loss:.4f})")
    else:
        epochs_no_improve += 1
        print(f" No improvement for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= patience:
        print(f"\n Early stopping at epoch {epoch+1}")
        break

# --- Restore best model before evaluation ---
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\n Restored model with best Train Loss = {best_loss:.4f}")

# === Predictions ===
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out, _ = model(data)  # predictions for all nodes
        out_target = out[data.target_node_ids].squeeze().cpu().numpy()
        # Ensure out_target is treated as an iterable even if it's a scalar
        if out_target.ndim == 0:
            out_target = out_target.reshape(1)
        predictions.extend(out_target)
        actuals.extend(data.y.cpu().numpy())

# === Inverse scaling and Metrics ===
aapl_scaler = joblib.load(f'scaler_{target_stock.lower()}.save')

predictions = aapl_scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
actuals = aapl_scaler.inverse_transform(np.array(actuals).reshape(-1, 1)).flatten()

rmse = np.sqrt(mean_squared_error(actuals, predictions))
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100
r2 = r2_score(actuals, predictions)

print(f"{target_stock} — RMSE: {rmse:.4f} | MAE: {mae:.4f} | MAPE: {mape:.2f}% | R²: {r2:.4f}")

# === Plotting ===
plt.figure(figsize=(10, 4))
plt.plot(actuals, label="Actual", color='black')
plt.plot(predictions, label="Predicted", color='red', alpha=0.7)
plt.title(f"{target_stock}: Actual vs Predicted Close Price (Over Time)")
plt.xlabel("Days")
plt.ylabel("Close Price")
plt.legend()
plt.grid(True)
plt.show()

gnn_predictions = predictions

# assume gnn_predictions is a numpy array (len = N)
gnn_predictions = np.array(gnn_predictions)

# Compare today vs yesterday
gnn_updown = np.zeros_like(gnn_predictions, dtype=int)

for i in range(1, len(gnn_predictions)):
    if gnn_predictions[i] > gnn_predictions[i-1]:
        gnn_updown[i] = 1  # up
    else:
        gnn_updown[i] = 0  # down

# Force the first value to be "up" as you requested
gnn_updown[0] = 1
gnn_nvda_updown = gnn_updown



# Extract attention weights for the target node (AAPL)
target_stock = 'NVDA'
# Assuming 'tickers' list and 'panel' dictionary are available from previous cells
tickers_in_panel = list(panel.keys())
aapl_node_index = tickers_in_panel.index(target_stock)

aapl_attention_weights = []

for edge_index, attn_weights in test_attn_weights:
    # Find edges where the target node is AAPL
    # In a fully connected graph, the target node is in the second row of edge_index
    # and its index is aapl_node_index
    aapl_edges_mask = edge_index[1] == aapl_node_index

    # Get the attention weights for these edges
    weights_to_aapl = attn_weights[aapl_edges_mask]
    aapl_attention_weights.append(weights_to_aapl)

# aapl_attention_weights is a list of tensors, where each tensor contains the attention
# weights from all other nodes towards AAPL for a given time step in the test set.

print(f"Extracted attention weights for {target_stock} for {len(aapl_attention_weights)} time steps.")
# You can inspect the shape of the weights for a single time step
if aapl_attention_weights:
    print(f"Shape of attention weights for one time step: {aapl_attention_weights[0].shape}")


# Calculate average attention weights across the test set
# Assuming aapl_attention_weights is a list of tensors where each tensor is [num_nodes, heads]
# We want to average across the time steps (the list) and potentially across heads

if aapl_attention_weights:
    # Concatenate all attention weight tensors from the list
    all_weights_tensor = torch.cat(aapl_attention_weights, dim=0) # Shape: [num_days * num_nodes, heads]

    # Reshape to [num_days, num_nodes, heads] and average over days
    num_days_test = len(aapl_attention_weights)
    num_nodes = all_weights_tensor.shape[0] // num_days_test
    num_heads = all_weights_tensor.shape[1]

    average_weights_per_node_head = all_weights_tensor.reshape(num_days_test, num_nodes, num_heads).mean(dim=0) # Shape: [num_nodes, heads]

    # Average across heads to get a single weight per node
    average_weights_per_node = average_weights_per_node_head.mean(dim=1) # Shape: [num_nodes]

    # Get the tickers corresponding to the nodes
    # Use tickers_to_process as the index, since the graph was built using these tickers

    # Create a pandas Series for easier sorting and visualization
    average_attention_series = pd.Series(average_weights_per_node.numpy(), index=tickers_in_panel) # Use corrected index

    # Sort the series by attention weight
    sorted_attention = average_attention_series.sort_values(ascending=False)

    # Plot the top N attention weights
    top_n = 10 # Adjusted top_n
    plt.figure(figsize=(12, 6))
    sorted_attention.head(top_n).plot(kind='bar')
    plt.title(f"Top {top_n} Average Attention Weights Towards {target_stock}")
    plt.xlabel("Ticker")
    plt.ylabel("Average Attention Weight")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    print(f"Top {top_n} tickers by average attention weight towards {target_stock}:")
    print(sorted_attention.head(top_n))

else:
    print("No attention weights were extracted.")

In [ ]:
# === Model Definition ===
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# === Load Graph Data ===
target_stock = 'SPY'
graph_list = torch.load(f"graph_list_{target_stock.lower()}_21day_rsimacd.pt", weights_only=False)

test_days = 252
train_graphs = graph_list[:-test_days]
test_graphs = graph_list

batch_size = 1
train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_graphs, batch_size=batch_size, shuffle=False)

# === Model Definition ===
class GATPredictor(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=4, dropout_prob=0.2):
        super(GATPredictor, self).__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.fc = nn.Linear(hidden_channels * heads, out_channels)  # Predict price for each node
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x, (edge_index_attn, attn_weights_tensor) = self.gat1(
            x, edge_index, return_attention_weights=True
        )
        x = F.elu(x)
        x = self.dropout(x)  # Apply dropout after activation
        output = self.fc(x)  # shape: [num_nodes, 1]
        return output, (edge_index_attn, attn_weights_tensor)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GATPredictor(in_channels=21, hidden_channels=32, out_channels=1).to(device) # Changed in_channels to 1
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
loss_fn = nn.MSELoss()

# === Training / Testing ===
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, _ = model(data)  # predictions for all nodes, shape [num_nodes, 1]
        out_target_nodes = out[data.target_node_ids].squeeze() # select and squeeze predictions for target nodes, shape [num_targets]
        # Ensure shapes are compatible for MSELoss, unsqueeze if scalar
        if out_target_nodes.ndim == 0:
            out_target_nodes = out_target_nodes.unsqueeze(0)
        if data.y.ndim == 0:
            data.y = data.y.unsqueeze(0)
        loss = loss_fn(out_target_nodes, data.y.to(device)) # compare with actual targets, shape [num_targets]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def test(loader):
    model.eval()
    total_loss = 0
    all_attn_weights = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out, (edge_index_attn, attn_weights_tensor) = model(data)  # predictions for all nodes, shape [num_nodes, 1]
            out_target_nodes = out[data.target_node_ids].squeeze() # select and squeeze predictions for target nodes, shape [num_targets]
            # Ensure shapes are compatible for MSELoss, unsqueeze if scalar
            if out_target_nodes.ndim == 0:
                out_target_nodes = out_target_nodes.unsqueeze(0)
            if data.y.ndim == 0:
                data.y = data.y.unsqueeze(0)
            loss = loss_fn(out_target_nodes, data.y.to(device)) # compare with actual targets, shape [num_targets]
            total_loss += loss.item()
            all_attn_weights.append((edge_index_attn.cpu(), attn_weights_tensor.cpu())) # Store edge_index and attention_weights_tensor

    return total_loss / len(loader), all_attn_weights

import copy

patience = 5   # how many epochs to wait for improvement
best_loss = float("inf")
epochs_no_improve = 0
best_model_state = None
n_epochs = 30  # or however many max you want

for epoch in range(n_epochs):
    train_loss = train()
    test_loss, test_attn_weights = test(test_loader)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early stopping check (based on train loss) ---
    if test_loss < best_loss - 1e-6:   # small delta to avoid floating point noise
        best_loss = test_loss
        best_model_state = copy.deepcopy(model.state_dict())
        epochs_no_improve = 0
        print(f" New best model saved (Train Loss: {best_loss:.4f})")
    else:
        epochs_no_improve += 1
        print(f" No improvement for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= patience:
        print(f"\n Early stopping at epoch {epoch+1}")
        break

# --- Restore best model before evaluation ---
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\n Restored model with best Train Loss = {best_loss:.4f}")

# === Predictions ===
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out, _ = model(data)  # predictions for all nodes
        out_target = out[data.target_node_ids].squeeze().cpu().numpy()
        # Ensure out_target is treated as an iterable even if it's a scalar
        if out_target.ndim == 0:
            out_target = out_target.reshape(1)
        predictions.extend(out_target)
        actuals.extend(data.y.cpu().numpy())

# === Inverse scaling and Metrics ===
aapl_scaler = joblib.load(f'scaler_{target_stock.lower()}.save')

predictions = aapl_scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
actuals = aapl_scaler.inverse_transform(np.array(actuals).reshape(-1, 1)).flatten()

rmse = np.sqrt(mean_squared_error(actuals, predictions))
mae = mean_absolute_error(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100
r2 = r2_score(actuals, predictions)

print(f"{target_stock} — RMSE: {rmse:.4f} | MAE: {mae:.4f} | MAPE: {mape:.2f}% | R²: {r2:.4f}")

# === Plotting ===
plt.figure(figsize=(10, 4))
plt.plot(actuals, label="Actual", color='black')
plt.plot(predictions, label="Predicted", color='red', alpha=0.7)
plt.title(f"{target_stock}: Actual vs Predicted Close Price (Over Time)")
plt.xlabel("Days")
plt.ylabel("Close Price")
plt.legend()
plt.grid(True)
plt.show()

gnn_predictions = predictions

# assume gnn_predictions is a numpy array (len = N)
gnn_predictions = np.array(gnn_predictions)

# Compare today vs yesterday
gnn_updown = np.zeros_like(gnn_predictions, dtype=int)

for i in range(1, len(gnn_predictions)):
    if gnn_predictions[i] > gnn_predictions[i-1]:
        gnn_updown[i] = 1  # up
    else:
        gnn_updown[i] = 0  # down

# Force the first value to be "up" as you requested
gnn_updown[0] = 1
gnn_spy_updown = gnn_updown

# Extract attention weights for the target node (AAPL)
target_stock = 'SPY'
# Assuming 'tickers' list and 'panel' dictionary are available from previous cells
tickers_in_panel = list(panel.keys())
aapl_node_index = tickers_in_panel.index(target_stock)

aapl_attention_weights = []

for edge_index, attn_weights in test_attn_weights:
    # Find edges where the target node is AAPL
    # In a fully connected graph, the target node is in the second row of edge_index
    # and its index is aapl_node_index
    aapl_edges_mask = edge_index[1] == aapl_node_index

    # Get the attention weights for these edges
    weights_to_aapl = attn_weights[aapl_edges_mask]
    aapl_attention_weights.append(weights_to_aapl)

# aapl_attention_weights is a list of tensors, where each tensor contains the attention
# weights from all other nodes towards AAPL for a given time step in the test set.

print(f"Extracted attention weights for {target_stock} for {len(aapl_attention_weights)} time steps.")
# You can inspect the shape of the weights for a single time step
if aapl_attention_weights:
    print(f"Shape of attention weights for one time step: {aapl_attention_weights[0].shape}")


# Calculate average attention weights across the test set
# Assuming aapl_attention_weights is a list of tensors where each tensor is [num_nodes, heads]
# We want to average across the time steps (the list) and potentially across heads

if aapl_attention_weights:
    # Concatenate all attention weight tensors from the list
    all_weights_tensor = torch.cat(aapl_attention_weights, dim=0) # Shape: [num_days * num_nodes, heads]

    # Reshape to [num_days, num_nodes, heads] and average over days
    num_days_test = len(aapl_attention_weights)
    num_nodes = all_weights_tensor.shape[0] // num_days_test
    num_heads = all_weights_tensor.shape[1]

    average_weights_per_node_head = all_weights_tensor.reshape(num_days_test, num_nodes, num_heads).mean(dim=0) # Shape: [num_nodes, heads]

    # Average across heads to get a single weight per node
    average_weights_per_node = average_weights_per_node_head.mean(dim=1) # Shape: [num_nodes]

    # Get the tickers corresponding to the nodes
    # Use tickers_to_process as the index, since the graph was built using these tickers

    # Create a pandas Series for easier sorting and visualization
    average_attention_series = pd.Series(average_weights_per_node.numpy(), index=tickers_in_panel) # Use corrected index

    # Sort the series by attention weight
    sorted_attention = average_attention_series.sort_values(ascending=False)

    # Plot the top N attention weights
    top_n = 10 # Adjusted top_n
    plt.figure(figsize=(12, 6))
    sorted_attention.head(top_n).plot(kind='bar')
    plt.title(f"Top {top_n} Average Attention Weights Towards {target_stock}")
    plt.xlabel("Ticker")
    plt.ylabel("Average Attention Weight")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    print(f"Top {top_n} tickers by average attention weight towards {target_stock}:")
    print(sorted_attention.head(top_n))

else:
    print("No attention weights were extracted.")

Concatenate each prediction:

In [ ]:
gnn_predictions = np.concatenate((gnn_aapl_updown, gnn_jpm_updown, gnn_nvda_updown, gnn_spy_updown))

Feed the concatenated predictions into a normal TFT. This code also produces the feature importance enncoder.

In [ ]:
import pandas as pd
import numpy as np
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.metrics import SMAPE
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from collections import defaultdict
import warnings
import logging
from ta.momentum import RSIIndicator
from ta.trend import MACD

warnings.filterwarnings("ignore")
logging.getLogger("lightning.pytorch.accelerators.cuda").setLevel(logging.WARNING)

# === Load and preprocess data for all stocks ===
def load_stock(file_path, stock_name, stock_or_etf):
    df = pd.read_csv(file_path, skiprows=3, header=None)
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df['group'] = stock_name
    df['stock_or_etf'] = stock_or_etf
    df['RSI'] = RSIIndicator(df['Close']).rsi()
    df['MACD'] = MACD(df['Close']).macd()
    df['upper_shadow'] = df['High'] - df[['Open', 'Close']].max(axis=1)
    df['lower_shadow'] = df[['Open', 'Close']].min(axis=1) - df['Low']
    return df

aapl = load_stock("AAPL.csv", "AAPL", "stock")
jpm  = load_stock("JPM.csv", "JPM", "stock")
nvda = load_stock("NVDA.csv", "NVDA", "stock")
spy  = load_stock("SPY.csv",  "SPY", "etf")

df = pd.concat([aapl, jpm, nvda, spy], ignore_index=True)
df = df[(df['Date'] > '2018-01-01') & (df['Date'] <= '2024-12-31')]
df = df.sort_values(['group', 'Date'])
df['time_idx'] = df.groupby('group').cumcount()
df['gnn_pred'] = gnn_predictions

# Split train/val and test (based on calendar year)
train_val_df = df[df['Date'] < "2024-01-01"].copy()
test_df = df[df['Date'] >= "2023-11-16"].copy()

# Recalculate time_idx in test_df based on prior max index
last_time_idx = train_val_df.groupby("group")["time_idx"].max().reset_index()
test_df = test_df.merge(last_time_idx, on="group", suffixes=("", "_last"))
test_df["time_idx"] = test_df.groupby("group").cumcount() + test_df["time_idx_last"] + 1
test_df.drop(columns=["time_idx_last"], inplace=True)

# === Define dataset parameters ===
max_encoder_length = 30
max_prediction_length = 1
training_cutoff = train_val_df["time_idx"].max() - 252  # leave last 252 days for validation

# === Training dataset ===
tft_dataset = TimeSeriesDataSet(
    train_val_df[train_val_df.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Close",
    group_ids=["group"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["Close", "RSI", "MACD", 'upper_shadow', 'lower_shadow', 'gnn_pred'],
    static_categoricals=["group", "stock_or_etf"],
    allow_missing_timesteps=True
)

# === Validation dataset (pre-2024 only) ===
val_df = train_val_df[(train_val_df.time_idx > (training_cutoff - max_encoder_length))]
validation = TimeSeriesDataSet.from_dataset(tft_dataset, val_df, stop_randomization=True)

train_loader = tft_dataset.to_dataloader(train=True, batch_size=32, num_workers=0)
val_loader = validation.to_dataloader(train=False, batch_size=32, num_workers=0)

# === Define LightningModule wrapper ===
class TFTLightningModule(LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        y_pred = out[0]
        loss = self.model.loss(y_pred, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=x['encoder_target'].shape[0])
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        y_pred = out[0]
        loss = self.model.loss(y_pred, y)
        self.log("val_loss", loss, prog_bar=True, batch_size=x['encoder_target'].shape[0])
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.model.hparams.learning_rate)

# === Train TFT model ===
tft = TemporalFusionTransformer.from_dataset(
    tft_dataset,
    learning_rate=0.003,
    hidden_size=32,
    attention_head_size=2,
    dropout=0.1,
    loss=SMAPE(),
    log_interval=10,
    reduce_on_plateau_patience=4,
).to("cuda")

module = TFTLightningModule(tft)

trainer = Trainer(
    max_epochs=20,
    gradient_clip_val=0.1,
    callbacks=[EarlyStopping(monitor="val_loss", patience=8, mode="min")],
    limit_train_batches=30,
    logger=TensorBoardLogger("lightning_logs")
)

trainer.fit(module, train_dataloaders=train_loader, val_dataloaders=val_loader)


# === Rolling prediction on 2024 test data ===
preds_by_group = defaultdict(list)
actuals_by_group = defaultdict(list)

for group in test_df["group"].unique():
    df_group = test_df[test_df["group"] == group].copy()
    df_group = df_group.reset_index(drop=True)
    df_group["time_idx"] = np.arange(len(df_group))  # reindex cleanly for this stock

    steps = len(df_group["time_idx"].unique()) - max_encoder_length - max_prediction_length

    for offset in range(steps):
        decoder_end_time = offset + max_encoder_length + max_prediction_length - 1
        data_slice = df_group[df_group.time_idx <= decoder_end_time].tail(max_encoder_length + max_prediction_length)

        if data_slice['time_idx'].nunique() < (max_encoder_length + max_prediction_length):
            continue

        try:
            predict_dataset = TimeSeriesDataSet.from_dataset(tft_dataset, data_slice, stop_randomization=True)
            predict_loader = predict_dataset.to_dataloader(train=False, batch_size=1, num_workers=0)

            prediction = tft.predict(predict_loader).cpu().numpy().flatten()[0]
            actual = data_slice.iloc[-1]["Close"]

            preds_by_group[group].append(prediction)
            actuals_by_group[group].append(actual)
        except Exception as e:
            print(f"Skipping {group} at offset {offset} due to error: {e}")


# === Evaluation & Plotting ===
for group in preds_by_group:
    preds = preds_by_group[group]
    acts = actuals_by_group[group]

    if len(preds) == 0 or len(acts) == 0:
        print(f"\nNot enough data to evaluate {group}. Skipping...")
        continue

    rmse = np.sqrt(mean_squared_error(acts, preds))
    mae = mean_absolute_error(acts, preds)
    r2 = r2_score(acts, preds)
    mape = np.mean(np.abs((np.array(acts) - np.array(preds)) / np.array(acts))) * 100

    print(f"\n {group} Evaluation:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE : {mae:.4f}")
    print(f"  R²  : {r2:.4f}")
    print(f"  MAPE: {mape:.2f}%")

    plt.figure(figsize=(12, 5))
    plt.plot(acts, label=f"{group} Actual")
    plt.plot(preds, label=f"{group} Predicted")
    plt.title(f"{group} - 1-Day Ahead Rolling Forecast (2024)")
    plt.xlabel("Trading Days")
    plt.ylabel("Close Price")
    plt.legend()
    plt.grid()
    plt.show()


# Select a representative sample from validation set
interpretation_dataset = TimeSeriesDataSet.from_dataset(
    tft_dataset,
    val_df.tail(200),  # adjust size if needed
    stop_randomization=True
)

interpretation_loader = interpretation_dataset.to_dataloader(train=False, batch_size=64, num_workers=0)

# Get raw predictions and attention weights
raw_output = tft.predict(interpretation_loader, mode="raw", return_x=True, return_index=True)

# `raw_output` is a dictionary-like object
raw_predictions = raw_output[0]
x = raw_output[1]

# Interpret attention / variable importance
interpretation = tft.interpret_output(raw_predictions, reduction="mean")

# Plot
tft.plot_interpretation(interpretation)